In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from keras import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import normalize

In [ ]:
df_train=pd.read_csv('../input/aicomp/train_data.csv')
df_train

In [ ]:
# df_train['strength']=df_train['strength']/df_train['strength'].mean()

Y=df_train['strength']
# df_train['cement']=df_train['cement']*2
# df_train['blast']=df_train['blast']*4
# df_train['flyash']=df_train['flyash']*4
# df_train['water']=df_train['water']*3
# df_train['sp']=df_train['sp']*30
# df_train['ca']=df_train['ca']/df_train['ca'].max()
# df_train['fa']=df_train['fa']/df_train['fa'].max()
# df_train['age']=df_train['age']/df_train['age'].max()
X_train = df_train[['cement', 'blast', 'flyash', 'sp', 'ca', 'fa', 'age']]
# scale = StandardScaler()
# X_train = scale.fit_transform(X_train)
# seed = 7
# X_train=normalize(X_train,axis=-1,order=2)
# Y=normalize(X_train,axis=-1,order=2)
print(X_train)
# X_train = (X_train.iloc[:,:].values).astype('float32')
# np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X_train, Y, test_size=0.3,shuffle=True,random_state=42)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train= scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# print(X_test)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

selector = SelectKBest(f_classif, k=7)
selected_features = selector.fit_transform(X_train,y_train)
f_score_indexes = (-selector.scores_).argsort()[:7]
f_score_indexes

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# seed = 7
# np.random.seed(seed)
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# cvscores = []
# for train, test in kfold.split(X_train, Y):
#     model = Sequential()
#     model.add(Dense(1000, input_dim=X_train.shape[1], activation='relu'))
#     model.add(Dropout(0.3))
#     model.add(Dense(500, activation='relu'))
#     model.add(Dense(250, activation='relu'))
#     model.add(Dense(1))
#     model.compile(loss='mean_squared_error', optimizer='adam', metrics=[metrics.mae])
#     model.fit(X[train], Y[train], epochs=1400, batch_size=32, verbose=0)
#     scores = model.evaluate(X[test], Y[test], verbose=0)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     cvscores.append(scores[1] * 100)
# print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
# from keras.layers import LeakyReLU

In [ ]:
#best model is obtained using trial and error.therefore, values can be different
def create_model():
    model = Sequential()
    model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.15))
    model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.15))
    model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.15))
    model.add(Dense(32, activation='relu'))
#     model.add(Dense(25, activation='relu'))
#     model.add(Dense(25, activation='relu'))
#     model.add(Dense(25, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(2000, activation='relu'))
#     model.add(Dense(1000, activation='relu'))
#     model.add(Dropout(0.2))
# #     model.add(Dropout(0.2))
#     model.add(Dense(400, activation='relu'))
#     model.add(Dense(200, activation='relu'))

    model.add(Dense(1))
    # Compile model
    model.compile(optimizer ='adam', loss = 'mse', 
              metrics =[metrics.RootMeanSquaredError()])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
history = model.fit(X_train, y_train.values, validation_data=(X_test,y_test.values), epochs=1000, batch_size=16)

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
df_que=pd.read_csv('../input/aicomp/test_data.csv')
y_ans=model.predict(df_que)
print(y_ans)
y_ans=y_ans.flatten()

In [ ]:
def write_preds(preds, fname):
    pd.DataFrame({"Id": list(range(0,len(preds))), "Predicted": preds}).to_csv(fname, index=False, header=True)

write_preds(y_ans, "comp10.csv")